##Installing Dependencies

In [18]:
%%capture installation_log
!pip install openai datasets scikit-learn matplotlib pandas -q

In [6]:
# Core Python libraries
from datasets import load_dataset
from pprint import pprint
import json
import pandas as pd

In [2]:
from google.colab import userdata
import os
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
hf_profile = 'aymangomaa'

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive


## Load Dataset

In [8]:
from datasets import load_dataset

dataset = load_dataset(f"{hf_profile}/entity_extraction_ade_v2_chat_base")
dataset

README.md:   0%|          | 0.00/718 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/771k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/92.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/99.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3458 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/385 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/428 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'relations', 'messages'],
        num_rows: 3458
    })
    validation: Dataset({
        features: ['text', 'relations', 'messages'],
        num_rows: 385
    })
    test: Dataset({
        features: ['text', 'relations', 'messages'],
        num_rows: 428
    })
})

In [9]:
test_data = dataset["test"]

In [10]:
pprint(test_data[9])

{'messages': [{'content': 'Extract all adverse drug effect (ADE) relationships '
                          'from the sentence. ### TEXT: METHODS: We report a '
                          'patient who had an anaphylactic reaction during the '
                          'intravenous infusion of cyclosporine.',
               'role': 'user'},
              {'content': '[{"ade": "anaphylactic reaction", "drug": '
                          '"cyclosporine"}]',
               'role': 'assistant'}],
 'relations': [{'ade': 'anaphylactic reaction', 'drug': 'cyclosporine'}],
 'text': 'METHODS: We report a patient who had an anaphylactic reaction during '
         'the intravenous infusion of cyclosporine.'}


In [11]:
# Extract Prompts from Dataset
prompts = [x['messages'][0]['content'] for x in test_data]  # extract prompts only
pprint(prompts[4])

('Extract all adverse drug effect (ADE) relationships from the sentence. ### '
 'TEXT: To the best of our knowledge, this is the first case of '
 'lithium-associated CDI and NDI presenting concurrently.')


## Inference with OpenAI APIs

In [28]:
from openai import OpenAI
OPENAI_AI_KEY = userdata.get('OPEN_AI_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_AI_KEY
client = OpenAI(api_key=OPENAI_AI_KEY)

In [33]:
from openai import OpenAI
from tqdm import tqdm
import json



def openai_zero_shot(test_data):
    if isinstance(test_data, dict):
        test_data = [test_data]

    predictions = []

    for example in tqdm(test_data, desc="OpenAI Zero-Shot Inference"):
        prompt = f"""
You are an expert biomedical relation extractor.

Extract all adverse drug effect (ADE) relationships from the sentence below.
Each relationship must be expressed as a dictionary with keys "drug" and "ade".
Return a valid JSON list of dictionaries.

Sentence: {example['text']}
"""

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=512,
            )
            content = response.choices[0].message.content.strip()
            parsed = json.loads(content)
        except Exception as e:
            print(f"Error processing: {example['text'][:60]}...\nReason: {e}")
            parsed = []
        predictions.append(parsed)

    return predictions

openai_preds = openai_zero_shot(test_data)


with open("openai_preds.json", "w") as f:
    json.dump(openai_preds, f, indent=2)


OpenAI Zero-Shot Inference: 100%|██████████| 428/428 [11:54<00:00,  1.67s/it]


In [34]:
import json
import re

# Load predictions
with open("openai_preds.json", "r") as f:
    openai_preds = json.load(f)

# Utility functions
def normalize(text):
    return re.sub(r"[^a-z0-9]", "", text.lower().strip())

def extract_pairs(rel_list):
    try:
        return set(
            (normalize(d.get("drug", "")), normalize(d.get("ade", "")))
            for d in rel_list
            if "drug" in d and "ade" in d
        )
    except:
        return set()

# Initialize counters
tp, fp, fn = 0, 0, 0

# Evaluate predictions
for i in range(len(test_data)):
    gold = extract_pairs(test_data[i]["relations"])
    pred = extract_pairs(openai_preds[i])

    tp += len(gold & pred)
    fp += len(pred - gold)
    fn += len(gold - pred)

# Compute scores
precision = tp / (tp + fp + 1e-8)
recall = tp / (tp + fn + 1e-8)
f1 = 2 * precision * recall / (precision + recall + 1e-8)

# Display results
print("📊 OpenAI GPT-4 ADE Extraction Evaluation")
print(f"TP: {tp}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


📊 OpenAI GPT-4 ADE Extraction Evaluation
TP: 396
FP: 227
FN: 272
Precision: 0.6356
Recall:    0.5928
F1 Score:  0.6135
